In [21]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
import folium

In [2]:
with open('kakao api.txt', mode = 'r') as key_file :
    kakao_key = key_file.read()
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'

In [52]:
park['면적'] = park['면적'].fillna(method = 'ffill')

In [53]:
park = pd.read_csv('서울시 주요 공원현황.csv', encoding = 'euckr')
park['면적'] = park['면적'].fillna(method = 'ffill')

lng = []
lat = []
addr = []
error = []

for i in park['공원주소'].index :
    try :
        url = f'{local_url}?query={quote(park["공원주소"][i])}'
        result = requests.get(url, headers = {"Authorization": f'KakaoAK {kakao_key}'}).json()
        for key, value in result['documents'][0].items() :
            if key == 'x' :
                lng.append(value)
            if key == 'y' : 
                lat.append(value)
            if key == 'address_name' :
                addr.append(value)
    except :
        print(i + 2)

In [59]:
data = {
    'name' : park['공원명'],
    'address' : addr,
    'latitude' : lat,
    'longitude' : par['면적']
    'area' : area
}
seoul_park = pd.DataFrame(data)
seoul_park.to_csv('seoul_park.csv', index = False, sep = ',')

SyntaxError: invalid syntax (<ipython-input-59-d7ae68d2baa9>, line 6)

In [ ]:
seoul_park = pd.read_csv('seoul_park.csv')
seoul_park

In [ ]:
folium.Circle(location = [37.5641201543296, 126.998009728978],
                radius = 100, tooltip = '중구청', color = 'crimson', fill = False).add_to(map)

In [32]:
seoul_park_map = folium.Map(location = [seoul_park['latitude'].mean(), seoul_park['longitude'].mean()], zoom_start = 11)
for i in seoul_park.index :
    folium.Marker(location = [seoul_park['latitude'][i], seoul_park['longitude'][i]],
                    popup = folium.Popup(seoul_park['address'][i], min_width = 200, max_width = 200),
                        tooltip = seoul_park['name'][i]).add_to(seoul_park_map)
seoul_park_map